In [ ]:
test1 = [1689159143, 1689159155, 1689159167, 1689159179, 1689159191, 1689159203, 1689159215, 1689159227, 1689159239, 1689159251, 1689159263, 1689159275, 1689159287, 1689159299, 1689159311, 1689159323, 1689159335, 1689159347, 1689159359, 1689159371, 1689159383, 1689159395, 1689159407, 1689159419, 1689159431, 1689159443, 1689159455, 1689159467, 1689159479, 1689159491, 1689159503, 1689159515]

test2 = [2706275214132550.0, 6852363096023834.0, 2132680071825315.5, 2439244865125883.0, 2918438452110884.5, 3084553320947596.0, 3117381892202906.0, 2282124708574623.5, 1868239646471544.8, 1892207340104222.0, 1892348964037182.2, 2435394967094290.5, 2418603244146521.0, 2097412258122442.8, 1908760340647254.2, 1317379839301867.2, 2780312655578977.0, 2673206193410480.5, 1878461746845751.2, 1977457224532762.0, 2418293411992915.5, 2871701402889452.5, 2561575993572513.5, 2874988020255183.5, 2190104192498386.8, 1446870362513832.2, 1941351147651972.2, 1370521158516176.0, 3092863006370522.0, 1787335736253756.8, 2453967147320570.5, 3894307579730440.0, 4877102391261606.0]

print(len(test1))

print(len(test2))



sorted_list = sorted(test1)
print(sorted_list)

In [1]:
test3 = [2223740781118001.0, 1260141498394935.2, 1833263316789592.5, 1487185910169561.2, 1868364700173924.5, 3608698475630962.0, 5147584386186307.0, 3712622497287841.0, 1742002882392932.8, 1971341639953858.8, 1704805814063778.8, 2948367506019705.5, 2671363233104959.5, 1994338944882766.2, 1755463848111533.8, 1464664204929831.5, 2239096091111167.0, 1882631023777761.2, 1348378269229444.5, 1675332964414718.2, 1886072499694509.0, 1501989395396047.8, 1920667160840891.5, 1511428925063234.2, 1508092499716823.5, 1671068731449896.8, 1876966221113261.2, 2970682537884551.0, 1683180892117103.8, 1777797346774442.2, 1719972934608254.2, 3656878057230982.0, 1759586217214666.5, 1579306425714490.8, 2500703828950420.0, 1386063347875264.2, 2717162267322661.5, 1796404576738850.8, 3909608836397385.0, 2836246115134082.5, 1544210926615698.2, 2160123343125648.5, 1153389627911318.2, 813911008522281.2, 1793739168558314.0, 1904259387241792.2, 1854302655587228.5, 1678227162829031.2, 1394634426323608.8, 1463291575638085.5, 3050290336036518.5, 1653091027461810.5, 1685415937397343.8, 1550906508241583.5, 3364484868623768.5, 1721190274968848.8, 1728616254839554.2, 1356165262407342.8, 2215845801907240.5, 2371549205913082.0, 1475772559208167.0, 1341761199917168.5, 1809715353821554.5, 1932637579393046.8, 1458145533435226.2, 1323914518788046.5, 1604581075926606.0, 1282123600206909.8, 1576425300413591.8, 6121612531600747.0, 2126250712852747.2, 1479716963974625.0, 1546268647810708.2, 1221432698020856.8, 7556580628353901.0, 2054814446990511.2, 1607139947781092.2, 1272740756169443.5, 1181262022666412.5, 1662643616331016.2, 1339424686646531.5, 1684433839727963.0, 2032036542684731.8, 3486548831279181.5, 1409833265697676.2, 1707375600834625.5, 1248064694568565.2, 1321249411548967.5, 1651356207760867.5, 1557627075825925.0, 2199627038890229.2, 1367511110547313.0, 1738163284024308.5, 1507299675042650.2, 1294566001225833.0, 1004694625812326.8, 1397119254705388.8, 1438048530630744.0, 1389885635399931.2, 2562658061662502.0, 2442664414506375.0]

print(len(test3))

101


In [2]:
import pyspark.sql.functions as F

from pyspark.sql import SparkSession

# Criação da sessão do Spark
spark = SparkSession.builder \
    .appName("Nome do App") \
    .getOrCreate()


avg_gas_media = [2223740781118001.0, 1260141498394935.2, 1833263316789592.5, 1487185910169561.2, 1868364700173924.5, 3608698475630962.0, 5147584386186307.0, 3712622497287841.0, 1742002882392932.8, 1971341639953858.8, 1704805814063778.8, 2948367506019705.5, 2671363233104959.5, 1994338944882766.2, 1755463848111533.8, 1464664204929831.5, 2239096091111167.0, 1882631023777761.2, 1348378269229444.5, 1675332964414718.2, 1886072499694509.0, 1501989395396047.8, 1920667160840891.5, 1511428925063234.2, 1508092499716823.5, 1671068731449896.8, 1876966221113261.2, 2970682537884551.0, 1683180892117103.8, 1777797346774442.2, 1719972934608254.2, 3656878057230982.0, 1759586217214666.5, 1579306425714490.8, 2500703828950420.0, 1386063347875264.2, 2717162267322661.5, 1796404576738850.8, 3909608836397385.0, 2836246115134082.5, 1544210926615698.2, 2160123343125648.5, 1153389627911318.2, 813911008522281.2, 1793739168558314.0, 1904259387241792.2, 1854302655587228.5, 1678227162829031.2, 1394634426323608.8, 1463291575638085.5, 3050290336036518.5, 1653091027461810.5, 1685415937397343.8, 1550906508241583.5, 3364484868623768.5, 1721190274968848.8, 1728616254839554.2, 1356165262407342.8, 2215845801907240.5, 2371549205913082.0, 1475772559208167.0, 1341761199917168.5, 1809715353821554.5, 1932637579393046.8, 1458145533435226.2, 1323914518788046.5, 1604581075926606.0, 1282123600206909.8, 1576425300413591.8, 6121612531600747.0, 2126250712852747.2, 1479716963974625.0, 1546268647810708.2, 1221432698020856.8, 7556580628353901.0, 2054814446990511.2, 1607139947781092.2, 1272740756169443.5, 1181262022666412.5, 1662643616331016.2, 1339424686646531.5, 1684433839727963.0, 2032036542684731.8, 3486548831279181.5, 1409833265697676.2, 1707375600834625.5, 1248064694568565.2, 1321249411548967.5, 1651356207760867.5, 1557627075825925.0, 2199627038890229.2, 1367511110547313.0, 1738163284024308.5, 1507299675042650.2, 1294566001225833.0, 1004694625812326.8, 1397119254705388.8, 1438048530630744.0, 1389885635399931.2, 2562658061662502.0, 2442664414506375.0]

df_spark = spark.createDataFrame([(1689160895           ,2.223740781118001E15 ),
(1689160907           ,1.2601414983949352E15),
(1689160919           ,1.8332633167895925E15),
(1689160931           ,1.4871859101695612E15),
(1689160943           ,1.8683647001739245E15),
(1689160955           ,3.608698475630962E15 ),
(1689160967           ,5.147584386186307E15 ),
(1689160979           ,3.712622497287841E15 ),
(1689160991           ,1.7420028823929328E15),
(1689161003           ,1.9713416399538588E15),
(1689161015           ,1.7048058140637788E15),
(1689161027           ,2.9483675060197055E15),
(1689161039           ,2.6713632331049595E15),
(1689161051           ,1.9943389448827662E15),
(1689161063           ,1.7554638481115338E15),
(1689161075           ,1.4646642049298315E15),
(1689161087           ,2.239096091111167E15 ),
(1689161099           ,1.8826310237777612E15),
(1689161111           ,1.3483782692294445E15),
(1689161123           ,1.6753329644147182E15),
(1689161135           ,1.886072499694509E15 ),
(1689161147           ,1.5019893953960478E15),
(1689161159           ,1.9206671608408915E15),
(1689161171           ,1.5114289250632342E15),
(1689161183           ,1.5080924997168235E15),
(1689161195           ,1.6710687314498968E15),
(1689161207           ,1.8769662211132612E15),
(1689161219           ,2.970682537884551E15 ),
(1689161231           ,1.6831808921171038E15),
(1689161243           ,1.7777973467744422E15),
(1689161255           ,1.7199729346082542E15),
(1689161267           ,3.656878057230982E15 ),
(1689161279           ,1.7595862172146665E15),
(1689161291           ,1.5793064257144908E15),
(1689161303           ,2.50070382895042E15  ),
(1689161315           ,1.3860633478752642E15),
(1689161327           ,2.7171622673226615E15),
(1689161339           ,1.7964045767388508E15),
(1689161351           ,3.909608836397385E15 ),
(1689161363           ,2.8362461151340825E15),
(1689161375           ,1.5442109266156982E15),
(1689161387           ,2.1601233431256485E15),
(1689161399           ,1.1533896279113182E15),
(1689161423           ,8.139110085222812E14 ),
(1689161435           ,1.793739168558314E15 ),
(1689161447           ,1.9042593872417922E15),
(1689161459           ,1.8543026555872285E15),
(1689161471           ,1.6782271628290312E15),
(1689161483           ,1.3946344263236088E15),
(1689161495           ,1.4632915756380855E15),
(1689161507           ,3.0502903360365185E15),
(1689161519           ,1.6530910274618105E15),
(1689161531           ,1.6854159373973438E15),
(1689161543           ,1.5509065082415835E15),
(1689161555           ,3.3644848686237685E15),
(1689161567           ,1.7211902749688488E15),
(1689161579           ,1.7286162548395542E15),
(1689161591           ,1.3561652624073428E15),
(1689161603           ,2.2158458019072405E15),
(1689161615           ,2.371549205913082E15 ),
(1689161627           ,1.475772559208167E15 ),
(1689161639           ,1.3417611999171685E15),
(1689161651           ,1.8097153538215545E15),
(1689161663           ,1.9326375793930468E15),
(1689161675           ,1.4581455334352262E15),
(1689161687           ,1.3239145187880465E15),
(1689161699           ,1.604581075926606E15 ),
(1689161711           ,1.2821236002069098E15),
(1689161723           ,1.5764253004135918E15),
(1689161735           ,6.121612531600747E15 ),
(1689161747           ,2.1262507128527472E15),
(1689161759           ,1.479716963974625E15 ),
(1689161771           ,1.5462686478107082E15),
(1689161783           ,1.2214326980208568E15),
(1689161795           ,7.556580628353901E15 ),
(1689161807           ,2.0548144469905112E15),
(1689161819           ,1.6071399477810922E15),
(1689161831           ,1.2727407561694435E15),
(1689161843           ,1.1812620226664125E15),
(1689161855           ,1.6626436163310162E15),
(1689161867           ,1.3394246866465315E15),
(1689161879           ,1.684433839727963E15 ),
(1689161891           ,2.0320365426847318E15),
(1689161903           ,3.4865488312791815E15),
(1689161915           ,1.4098332656976762E15),
(1689161927           ,1.7073756008346255E15),
(1689161939           ,1.2480646945685652E15),
(1689161951           ,1.3212494115489675E15),
(1689161963           ,1.6513562077608675E15),
(1689161975           ,1.557627075825925E15 ),
(1689161987           ,2.1996270388902292E15),
(1689161999           ,1.367511110547313E15 ),
(1689162011           ,1.7381632840243085E15),
(1689162023           ,1.5072996750426502E15),
(1689162035           ,1.294566001225833E15 ),
(1689162047           ,1.0046946258123268E15),
(1689162059           ,1.3971192547053888E15),
(1689162071           ,1.438048530630744E15 ),
(1689162083           ,1.3898856353999312E15),
(1689162095           ,2.562658061662502E15 )],
                                ["Timestamp_Transaction", "avg(Transaction_Fee)"])

# Criação do DataFrame a partir da lista avg_gas_media
avg_gas_media_df = spark.createDataFrame([(value,) for value in test3], ["avg(Transaction_Fee)"])

# Renomear a coluna "avg(Transaction_Fee)" em df_spark
df_spark_renamed = df_spark.withColumnRenamed("avg(Transaction_Fee)", "avg_transaction_fee")

# Coletar os valores distintos de avg_transaction_fee em df_spark_renamed
avg_transaction_fee_list = [row.avg_transaction_fee for row in df_spark_renamed.select("avg_transaction_fee").distinct().collect()]

# Filtragem dos valores não presentes em df_spark
filtered_values = avg_gas_media_df.filter(~F.col("avg(Transaction_Fee)").isin(avg_transaction_fee_list))

# Exibição dos valores filtrados
filtered_values.show()

+--------------------+
|avg(Transaction_Fee)|
+--------------------+
|2.442664414506375E15|
+--------------------+

